# Extraction de l'encapsulation de toutes les couches de l'encodeur de BERT
Nous avons appris à extraire les encastrements de BERT pré-entraînée dans la section précédente. Nous avons appris qu'il s'agit des encastrements obtenus à partir de la dernière couche de codage. La question est maintenant de savoir si nous devons considérer l'intégration obtenue uniquement à partir de la dernière couche de codage (état caché final) ou si nous devons également considérer l'intégration obtenue à partir de toutes les couches de codage (tous les états cachés). Examinons cette question plus en détail.



In [ ]:

!pip install transformers

In [6]:
from transformers import BertModel, BertTokenizer
import torch

Ensuite, téléchargez le modèle BERT pré-entraîné et le tokenizer. Comme nous pouvons le remarquer lors du téléchargement du modèle BERT pré-entraîné. Nous devons définir output_hidden_states = True. En mettant cette valeur à true, nous obtenons des embeddings de toutes les couches de l'encodeur :

In [7]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Ensuite, nous traitons les données d'entrée avant de les transmettre au modèle.

## Prétraitement de l'entrée
Considérons la même phrase que dans la section précédente. Tout d'abord, nous tokenisons la phrase et ajoutons le token [CLS] au début et le token [SEP] à la fin :

In [8]:
sentence = 'I love Paris'
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']

Supposons que nous devions maintenir la longueur des jetons à 7. Nous ajoutons donc les jetons [PAD] et définissons également le masque d'attention :

In [9]:
tokens = tokens + ['[PAD]'] + ['[PAD]']
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]

Ensuite, nous convertissons les tokens en token_ids :

In [10]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)

Nous convertissons maintenant les token_ids et les attention_mask en tenseurs :

In [11]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

Maintenant que nous avons prétraité l'entrée, obtenons l'intégration.

## Obtention de l'encapsulation
Puisque nous avons défini output_hidden_states = True lors de la définition du modèle pour obtenir les embeddings de toutes les couches de l'encodeur, le modèle renvoie maintenant un tuple de sortie avec trois valeurs comme indiqué ci-dessous :

In [17]:
outputs= model(token_ids, attention_mask = attention_mask)
last_hidden_state, pooler_output, hidden_states =outputs.last_hidden_state,outputs.pooler_output,outputs.hidden_states

Dans le code précédent, ce qui suit s'applique :

La première valeur last_hidden_state contient la représentation de tous les jetons obtenus uniquement à partir de la couche finale du codeur (codeur 12).
Ensuite, pooler_output indique la représentation du jeton [CLS] provenant de la dernière couche de codage, qui est ensuite traitée par une fonction d'activation linéaire et tanh.
hidden_states contient la représentation de tous les jetons obtenus à partir de toutes les couches de codage finales.
Examinons maintenant chacune de ces valeurs et comprenons-les plus en détail.

Tout d'abord, examinons last_hidden_state. Comme nous l'avons appris, elle contient la représentation de tous les tokens obtenus uniquement à partir de la couche finale de l'encodeur (encodeur 12). Imprimons la forme de last_hidden_state :

In [18]:
last_hidden_state.shape

torch.Size([1, 7, 768])

La taille [1,7,768] indique [batch_size, sequence_length, hidden_size].

La taille de notre lot est 1, la longueur de la séquence est la longueur du jeton et comme nous avons 7 jetons, la longueur de la séquence est 7, et la taille cachée est la taille de la représentation (embedding) et elle est de 768 pour le modèle BERT-base.

Nous pouvons obtenir l'intégration de chaque jeton comme suit :
- last_hidden[0][0] donne la représentation du premier token qui est [CLS]
- last_hidden[0][1] donne la représentation du deuxième jeton qui est 'I'.
- last_hidden[0][2] donne la représentation du troisième jeton qui est 'love'

De la même manière, nous pouvons obtenir la représentation de tous les jetons à partir de la couche finale de l'encodeur.

Ensuite, nous avons pooler_output qui contient la représentation du jeton [CLS] de la couche finale de l'encodeur qui est ensuite traitée par une fonction d'activation linéaire et tanh. Imprimons la forme du pooler_output :

In [19]:
pooler_output.shape

torch.Size([1, 768])

La taille [1 768] indique la [batch_size, hidden_size].

Nous avons appris que le jeton [CLS] contient la représentation agrégée de la phrase. Ainsi, nous pouvons utiliser le pooler_output comme représentation de la phrase donnée "J'aime Paris".

Enfin, nous avons hidden_states qui contient la représentation de tous les tokens obtenus à partir de toutes les couches finales de l'encodeur. Il s'agit d'un tuple contenant 13 valeurs représentant toutes les couches de codage (couches cachées), depuis la couche d'intégration d'entrée jusqu'à la couche de codage finale.

In [20]:
len(hidden_states)

13

Comme nous pouvons le constater, il contient 13 valeurs représentant toutes les couches. Ainsi :

- hidden_states[0] contient la représentation de tous les tokens obtenus à partir de la couche d'encodage d'entrée
- hidden_states[1] contient la représentation de tous les jetons obtenus à partir de la première couche d'encodage
- hidden_states[2] contient la représentation de tous les jetons obtenus à partir de la deuxième couche de codage.

De même, hidden_states[12] contient la représentation de tous les jetons obtenus à partir de la dernière couche de codage.
Explorons cela plus en détail. Tout d'abord, imprimons la forme de hidden_states[0] qui contient la représentation de tous les tokens obtenus à partir de la couche d'encodage d'entrée :

In [21]:
hidden_states[0].shape

torch.Size([1, 7, 768])

La taille [1,7,768] indique les [batch_size, sequence_length, hidden_size].

Imprimons maintenant la forme de hidden_states[1] qui contient la représentation de tous les tokens obtenus à partir de la première couche d'encodage :

In [22]:
torch.Size([1, 7, 768])

torch.Size([1, 7, 768])

In [23]:
hidden_states[1].shape

torch.Size([1, 7, 768])